In [1]:
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader, Subset
import torch
from torch import nn
import numpy as np
import matplotlib.pyplot as plt
import torch.optim as optim
from torch.amp import GradScaler, autocast
import os
import random
from torch.utils.data import DataLoader, SubsetRandomSampler
from collections import Counter

In [2]:
import torch
import numpy as np
from torch.utils.data import DataLoader, Subset
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import random

# Define dataset root directory
mnist_root = '/home/j597s263/scratch/j597s263/Datasets/MNIST'

random.seed(42)
torch.manual_seed(42)
np.random.seed(42)

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  
    transforms.ToTensor()
])

train_dataset = datasets.MNIST(root=mnist_root, transform=transform, train=True, download=False)
test_dataset = datasets.MNIST(root=mnist_root, transform=transform, train=False, download=False)

train_indices = list(range(len(train_dataset)))
random.shuffle(train_indices)  

split_idx = int(0.9 * len(train_indices))  
train_indices, attack_indices = train_indices[:split_idx], train_indices[split_idx:]

train_data = Subset(train_dataset, train_indices)
attack_data = Subset(train_dataset, attack_indices)

train_loader = DataLoader(train_data, batch_size=1, shuffle=True)  # Shuffle within batches
attack_loader = DataLoader(attack_data, batch_size=1, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

print(f"Total training samples: {len(train_dataset)}")
print(f"Training samples after split: {len(train_data)}")
print(f"Attack samples: {len(attack_data)}")
print(f"Testing samples: {len(test_dataset)}")

Total training samples: 60000
Training samples after split: 54000
Attack samples: 6000
Testing samples: 10000


In [3]:
import torch.nn as nn

# Residual block
class Residual(nn.Module):
    def __init__(self, fn):
        super().__init__()
        self.fn = fn

    def forward(self, x):
        return self.fn(x) + x

# ConvMixer model with hard-coded parameters
def ConvMixer():
    dim = 256          # Embedding dimension
    depth = 8          # Number of ConvMixer blocks
    kernel_size = 5    # Kernel size for depthwise convolution
    patch_size = 4     # Patch size for initial convolution
    n_classes = 10    # CIFAR-10 has 10 classes

    return nn.Sequential(
        nn.Conv2d(1, dim, kernel_size=patch_size, stride=patch_size),
        nn.GELU(),
        nn.BatchNorm2d(dim),
        *[nn.Sequential(
                Residual(nn.Sequential(
                    nn.Conv2d(dim, dim, kernel_size, groups=dim, padding="same"),
                    nn.GELU(),
                    nn.BatchNorm2d(dim)
                )),
                nn.Conv2d(dim, dim, kernel_size=1),
                nn.GELU(),
                nn.BatchNorm2d(dim)
        ) for _ in range(depth)],
        nn.AdaptiveAvgPool2d((1, 1)),
        nn.Flatten(),
        nn.Linear(dim, n_classes)
    )

In [4]:
# Load the model
import torch

# Define the path to the model
device = "cuda" 

# Load the model
model = torch.load('/home/j597s263/scratch/j597s263/Models/ConvModels/Base/ConvMNIBase.mod', weights_only=False, map_location="cuda")
model = model.to(device)
model.eval()  

print("Model loaded successfully!")

Model loaded successfully!


In [5]:
import torch
import numpy as np
from captum.attr import IntegratedGradients
import os

# Ensure the model is in evaluation mode and on the correct device
device = 'cuda'
model.to(device)
model.eval()

# Define the Integrated Gradients method
ig = IntegratedGradients(model)

# Function to compute and save explanations as .npy
def compute_and_save_explanations_with_labels(data_loader, save_dir):
    """
    Compute Integrated Gradient explanations for all images in the dataset and save as .npy files.

    Args:
        data_loader (DataLoader): DataLoader for the dataset to explain.
        save_dir (str): Directory to save the explanations as .npy files.
    """
    os.makedirs(save_dir, exist_ok=True)  # Ensure the directory exists

    for batch_idx, (images, labels) in enumerate(data_loader):
        images, labels = images.to(device), labels.to(device)

        # Compute attributions using IG
        attributions, delta = ig.attribute(
            inputs=images,
            target=labels,  # Use ground-truth labels for explanations
            baselines=torch.zeros_like(images).to(device),  # Baseline: black image
            return_convergence_delta=True,
            n_steps = 10
        )

        # Save explanations for each image in the batch
        for i in range(images.size(0)):
            label = labels[i].item()  # Get the label as an integer
            explanation = attributions[i].cpu().numpy()  # Shape: (1, 224, 224)

            # Create a label array of shape (1, 224, 224)
            label_array = np.full((1, explanation.shape[1], explanation.shape[2]), label, dtype=np.float32)

            # Concatenate label and explanation to shape (2, 224, 224) (for grayscale images)
            explanation_with_label = np.concatenate([label_array, explanation], axis=0)

            # Save as .npy
            save_path = os.path.join(save_dir, f"explanation_{batch_idx * images.size(0) + i}.npy")
            np.save(save_path, explanation_with_label)

        print(f"Processed batch {batch_idx + 1}/{len(data_loader)}")

    print(f"Explanations saved to {save_dir}")
    
# Define the directory to save explanations
save_dir = "/home/j597s263/scratch/j597s263/Datasets/Explanation_values/Conv/IG_ConvMNI"

# Compute and save explanations
compute_and_save_explanations_with_labels(attack_loader, save_dir)

Processed batch 1/6000
Processed batch 2/6000
Processed batch 3/6000
Processed batch 4/6000
Processed batch 5/6000
Processed batch 6/6000
Processed batch 7/6000
Processed batch 8/6000
Processed batch 9/6000
Processed batch 10/6000
Processed batch 11/6000
Processed batch 12/6000
Processed batch 13/6000
Processed batch 14/6000
Processed batch 15/6000
Processed batch 16/6000
Processed batch 17/6000
Processed batch 18/6000
Processed batch 19/6000
Processed batch 20/6000
Processed batch 21/6000
Processed batch 22/6000
Processed batch 23/6000
Processed batch 24/6000
Processed batch 25/6000
Processed batch 26/6000
Processed batch 27/6000
Processed batch 28/6000
Processed batch 29/6000
Processed batch 30/6000
Processed batch 31/6000
Processed batch 32/6000
Processed batch 33/6000
Processed batch 34/6000
Processed batch 35/6000
Processed batch 36/6000
Processed batch 37/6000
Processed batch 38/6000
Processed batch 39/6000
Processed batch 40/6000
Processed batch 41/6000
Processed batch 42/6000
P